# 0.0 Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1.0 Tratamento de dados faltantes

In [2]:
df_raw = pd.read_csv('../data/raw_data/df_raw.csv')
target_col = 'inadipl_90dias_ult2anos'

print("=" * 80)
print("🔍 ESTRATÉGIA DE TRATAMENTO DE DADOS FALTANTES")
print("=" * 80)

missing_analysis = pd.DataFrame({
    'Feature': df_raw.columns,
    'Missing_Count': df_raw.isnull().sum(),
    'Missing_Pct': (df_raw.isnull().sum() / len(df_raw) * 100).round(2),
    'Dtype': df_raw.dtypes
})

missing_analysis = missing_analysis[missing_analysis['Missing_Count'] > 0]
missing_analysis = missing_analysis.sort_values('Missing_Pct', ascending=False)

print(missing_analysis.to_string())

# Decisão de tratamento
print("\n📋 ESTRATÉGIA POR COLUNA:")
for idx, row in missing_analysis.iterrows():
    col = row['Feature']
    pct = row['Missing_Pct']
    
    if pct > 50:
        strategy = "❌ REMOVER coluna (>50% missing)"
    elif pct > 20:
        strategy = "⚠️ Criar flag + imputar ou remover"
    elif pct > 5:
        strategy = "📊 Imputar pela mediana/moda"
    else:
        strategy = "✅ Imputar pela mediana/moda"
    
    print(f"  {col:30s} ({pct:5.2f}%) -> {strategy}")

🔍 ESTRATÉGIA DE TRATAMENTO DE DADOS FALTANTES
                                     Feature  Missing_Count  Missing_Pct    Dtype
comprometimento_renda  comprometimento_renda          29749        19.83  float64
renda_mensal                    renda_mensal          29731        19.82  float64
faixa_etaria                    faixa_etaria            688         0.46      str
utilizacao_credito        utilizacao_credito            114         0.08  float64
divida_ratio                    divida_ratio             18         0.01  float64

📋 ESTRATÉGIA POR COLUNA:
  comprometimento_renda          (19.83%) -> 📊 Imputar pela mediana/moda
  renda_mensal                   (19.82%) -> 📊 Imputar pela mediana/moda
  faixa_etaria                   ( 0.46%) -> ✅ Imputar pela mediana/moda
  utilizacao_credito             ( 0.08%) -> ✅ Imputar pela mediana/moda
  divida_ratio                   ( 0.01%) -> ✅ Imputar pela mediana/moda


## 1.1 Imputação de valores segmentada por faixa etária (mediana dentro de cada grupo etário)

In [3]:
df_clean = df_raw.copy()

# Flags de missing (variáveis críticas)
df_clean['renda_mensal_missing'] = df_clean['renda_mensal'].isna().astype(int)
df_clean['comprometimento_renda_missing'] = df_clean['comprometimento_renda'].isna().astype(int)

# Imputação segmentada por faixa etária
for col in ['renda_mensal', 'comprometimento_renda']:
    df_clean[col] = df_clean.groupby('faixa_etaria')[col].transform(
        lambda x: x.fillna(x.median())
    )
    # Se ainda restarem NaN (grupos sem mediana), imputar com a mediana geral
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Faixa etária: imputar como "não informado"
df_clean['faixa_etaria'] = df_clean['faixa_etaria'].fillna('não informado')

# Variáveis com baixo missing → imputação simples
df_clean['utilizacao_credito'] = df_clean['utilizacao_credito'].fillna(df_clean['utilizacao_credito'].median())
df_clean['divida_ratio'] = df_clean['divida_ratio'].fillna(df_clean['divida_ratio'].median())
df_clean.isnull().sum()

id                               0
inadipl_90dias_ult2anos          0
utilizacao_credito               0
idade                            0
atrasos_30dias                   0
divida_ratio                     0
renda_mensal                     0
linhas_credito_abertas           0
atrasos_90dias                   0
emprestimos_imobiliarioss        0
dependentes                      0
comprometimento_renda            0
faixa_etaria                     0
renda_mensal_missing             0
comprometimento_renda_missing    0
dtype: int64

## 1.2 Tratamento de outliers

In [4]:
print("="*80)
print("🔧 TRATAMENTO DE OUTLIERS ORIENTADO AO NEGÓCIO DE CRÉDITO")
print("="*80)

# Idade fora de faixa (regras de elegibilidade)
df_clean.loc[df_clean['idade'] < 18, 'idade'] = 18
df_clean.loc[df_clean['idade'] > 90, 'idade'] = 90
print("✅ Idade truncada para [18, 90]")

# Comprometimento de renda (limite de aceitação orientado ao negócio)
# Até 30% = saudável, 30–50% = alerta, >50% = alto risco, >100% = crítico
df_clean.loc[df_clean['comprometimento_renda'] > 100, 'comprometimento_renda'] = 100
print("✅ Comprometimento de renda truncado em 100% (inadimplência crítica)")

# Winsorization para renda mensal (evitar valores absurdos)
# Regras de negócio: mínimo ~ salário mínimo, máximo ~ R$ 50 mil
renda_min = 1412   # salário mínimo 2026
renda_max = 50000  # limite plausível para PF em crédito massificado
df_clean['renda_mensal'] = np.clip(df_clean['renda_mensal'], renda_min, renda_max)
print(f"✅ Renda mensal truncada entre {renda_min:.2f} e {renda_max:.2f}")

# Opcional: sinalizar registros alterados
df_clean['idade_outlier_flag'] = ((df_raw['idade'] < 18) | (df_raw['idade'] > 90)).astype(int)
df_clean['comprometimento_outlier_flag'] = (df_raw['comprometimento_renda'] > 100).astype(int)
df_clean['renda_outlier_flag'] = ((df_raw['renda_mensal'] < renda_min) | (df_raw['renda_mensal'] > renda_max)).astype(int)

print("\n✅ Flags de outliers criadas para auditoria/modelagem")


🔧 TRATAMENTO DE OUTLIERS ORIENTADO AO NEGÓCIO DE CRÉDITO
✅ Idade truncada para [18, 90]
✅ Comprometimento de renda truncado em 100% (inadimplência crítica)
✅ Renda mensal truncada entre 1412.00 e 50000.00

✅ Flags de outliers criadas para auditoria/modelagem


In [5]:
df_clean.describe()

,id,utilizacao_credito,idade,atrasos_30dias,divida_ratio,renda_mensal,linhas_credito_abertas,atrasos_90dias,emprestimos_imobiliarioss,dependentes,comprometimento_renda,renda_mensal_missing,comprometimento_renda_missing,idade_outlier_flag,comprometimento_outlier_flag,renda_outlier_flag
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,75000.500000,6.048555,52.285007,0.421033,353.005119,6308.142220,8.452760,0.265973,1.018240,2.950060,93.094719,0.198207,0.198327,0.003267,0.707880,0.047060
std,43301.414527,249.755368,14.743264,4.192781,2037.818516,4551.263218,5.145951,4.169304,1.129771,39.129022,23.155866,0.398650,0.398741,0.057062,0.454739,0.211768
min,1.000000,0.000000,18.000000,0.000000,0.000000,1412.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,37500.750000,0.030000,41.000000,0.000000,0.175200,3876.000000,5.000000,0.000000,0.000000,0.000000,100.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,75000.500000,0.154600,52.000000,0.000000,0.366500,5366.000000,8.000000,0.000000,1.000000,0.000000,100.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,112500.250000,0.559025,63.000000,0.000000,0.868300,7382.000000,11.000000,0.000000,2.000000,2.000000,100.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,150000.000000,50708.000000,90.000000,98.000000,329664.000000,50000.000000,58.000000,98.000000,54.000000,985.000000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [13]:
# Salvando em excel .csv
import os

caminho_dir = r"C:\Users\wesle\anaconda3\envs\Credit_Score_Project\data\processed_data"
os.makedirs(caminho_dir, exist_ok=True)

df_clean.to_csv(os.path.join(caminho_dir, "df_cleaned.csv"), index=False, encoding="utf-8-sig")
print(f"   Shape original: {df_raw.shape}")
print(f"   Shape limpo: {df_clean.shape}")

   Shape original: (150000, 13)
   Shape limpo: (150000, 18)
